<center><img alt="" src="images/Cover_NLPTM.jpg"/></center> 

## <center><font color="blue">Text Analytics</font></center>
<b><center>(C) Taufik Sutanto - 2020</center>
<center>tau-data Indonesia ~ https://tau-data.id ~ taufik@tau-data.id</center>

## <center><font color="blue">Text Analytics</font></center>
<img alt="" src="images/PDS_logo.jpg" />

In [ ]:
# Installing Modules for Google Colab
import nltk

!wget https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/taudataNlpTm.py
!mkdir data
!wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/slang.txt
!wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/stopwords_id.txt
!wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/stopwords_en.txt
!wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/wn-ind-def.tab
!wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/wn-msa-all.tab
!wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/all_indo_man_tag_corpus_model.crf.tagger

!pip install spacy python-crfsuite unidecode textblob sastrawi tweepy twython
!python -m spacy download en
!python -m spacy download xx
!python -m spacy download en_core_web_sm

nltk.download('popular')

In [2]:
import taudataNlpTm as tau, seaborn as sns; sns.set()
import tweepy, json, nltk, urllib.request
from textblob import TextBlob
from nltk.tokenize import TweetTokenizer
from twython import TwythonStreamer  
from tqdm import tqdm_notebook as tqdm

# Data Gathering: Crawling, Streaming, Scrapping

In [ ]:
# Contoh API Keys (Sesuaikan dengan API keys masing-masing)
Ck = '' # consumer_key
Cs = '' # consumer_secret
At = '' # access_token
As = '' # access_secret
'Done'

In [5]:
def twitter_connect(Ck, Cs, At, As, verbose = 0):
    try:
        auth = tweepy.OAuthHandler(Ck, Cs)
        auth.set_access_token(At, As)
        twitter = tweepy.API(auth, timeout=120)
        if verbose != 0:
            user = twitter.verify_credentials()
            print('Welcome "%s" you are now connected to twitter server' %user.name)
        return twitter
    except:
        print("Connection failed, please check your API keys or connection")
        return None

In [6]:
# Koneksi ke twitter
twitter = twitter_connect(Ck, Cs, At, As, verbose = 1)

Welcome "Taufik Sutanto" you are now connected to twitter server


In [7]:
topic = 'psbb jakarta'
N = 100 # jumlah tweet yang ingin diambil
bahasa = 'id'
T = twitter.search(q=topic, lang=bahasa, count=N, tweet_mode = 'extended')
tweets = [t._json for t in T]
print(' Berhasil mendapatkan {} tweets'.format(len(tweets)))

 Berhasil mendapatkan 100 tweets


In [8]:
last_id = T[-1]._json['id'] - 1
T = twitter.search(q=topic, lang=bahasa, count=N, tweet_mode = 'extended', max_id=last_id)
tweets.extend([t._json for t in T])

print('Jumlah data sekarang = ', len(tweets))

Jumlah data sekarang =  200


In [14]:
D = [t['full_text'] for t in tweets]
D[:3]

['@Dennysiregar7 @mohmahfudmd Tak ada wibawa negara sedikitpun..Aturan hanya berlaku bila ada 1 sampai 10 org, tapi kalo udah ratusan org, melempem. Apa artinya aturan yg ada di PSBB yg sedang berlaku di DKI jakarta dan Tangerang.Sungguh lemah pemerintah ini.Apa aturan di PSBB bila baru datang ke indonesia.',
 '@AhlulQohwah PSBB jakarta berakhir 22 Nopember, besar kemungkinan 2-12 akan diizinkan dan digelar di monas, dan setelah acara 212 diterapakan lagi PSBB, memang dasar culas 1 orang ini, tanpa menghiraukan keselamatan warganya. Naudzu Billaah',
 'RT @RD_4WR1212: Menanggapi hal itu, Wakil Gubernur DKI Jakarta Ahmad Riza Patria tdk mau meminta langsung kpd Rizieq utk isolasi mandiri. S…']

# Sedikit Preprocessing

In [ ]:
stopId, lemmaId = tau.LoadStopWords(lang='id') 

In [17]:
data = []
for i, d in tqdm(enumerate(D)):
    doc = tau.cleanText(d, lemma=lemmaId, stops = stopId, symbols_remove = True, min_charLen = 3, fixTag= True)
    data.append(doc)
print(data[:3])


['dennysiregar7 mohmahfudmd wibawa negara sedikitpun aturan berlaku org kalo udah ratusan org melempem aturan psbb berlaku dki jakarta tangerang.sungguh lemah pemerintah ini.apa aturan psbb indonesia', 'ahlulqohwah psbb jakarta nopember diizinkan digelar monas acara 212 diterapakan psbb dasar culas orang menghiraukan keselamatan warganya naudzu billaah', 'rd_4wr1212 menanggapi wakil gubernur dki jakarta ahmad riza patria tdk langsung kpd rizieq utk isolasi mandiri ']


In [18]:
# Pertama-tama kita perlu menyimpan tweet yang telah dilakukan preprocessing diatas sebagai file teks biasa
# Menggunakan cara yang dibahas di hari pertama
file = 'data/dataTweet.txt'
with open(file, 'w') as f:
    for d in data:
        f.write(d+'\n')
'Done'

'Done'

<h2>Text Analytics</h2>

<ul>
	<li>Tidak seperti data terstruktur, data tidak terstruktur seperti teks termasuk salah satu data yang cukup sulit untuk divisualisasikan.<br />
	<img alt="" src="images/11_charts.jpg" style="height:150px; width:276px" /></li>
	<li>Namun terdapat Tools seperti Voyant yang dapat membantu dalam visualisasi sekaligus analisis.<br />
	<img alt="" src="images/11_voyant.png" style="height:118px; width:426px" /></li>
</ul>


<h3 id="Voyant-dapat-digunakan-dalam-2-cara:">Voyant dapat digunakan dalam 2 cara:</h3>

<ol>
	<li>
	<p><strong>Online</strong>:&nbsp;<a href="https://voyant-tools.org/" target="_blank">https://voyant-tools.org/</a><br />
	<u>Kelebihan</u>: Sederhana &amp; portable, tanpa harus install di komputer kita.<br />
	<u>Kekurangan</u>: butuh koneksi internet, tidak cocok untuk data teks yang besar, privacy.</p>
	</li>
	<li>
	<p><strong>Offline </strong>di komputer kita [Java Based]</p>
	</li>
</ol>


<h3 id="Instalasi-Voyant:">Instalasi Voyant:</h3>

<ol>
	<li>Unduh Voyant dari &nbsp;<a href="https://github.com/sgsinclair/VoyantServer" target="_blank">https://github.com/sgsinclair/VoyantServer</a></li>
	<li>Extract Voyant ke sembarang folder (disarankan <strong>C:\VoyantServer</strong>)</li>
	<li>Unduh Java JDK dari &nbsp;<a href="http://www.oracle.com/technetwork/java/javase/downloads/index.html&amp;nbsp" target="_blank">http://www.oracle.com/technetwork/java/javase/downloads/index.html&amp;nbsp</a>;</li>
	<li>Install Java</li>
	<li>Tambahkan variable <strong>Java Home</strong> ke &quot;System Variable&quot;</li>
	<li>Jalankan VoyantServer.jar dari&nbsp;C:\VoyantServer<br />
	<strong>Tips</strong>: Jalankan Voyant Server sebelum Jupyter atau ganti &quot;port&quot; di VVoyant Server.</li>
</ol>

<p><img alt="" src="images/JavaPath.PNG" style="width: 275px ; height: 300px" /></p>


# Kita akan menggunakan data Tweet diatas ( variable **"D"**)

## Namun sebelumnya preprocessing dilakukan pada data Text (tweet) tersebut
## Menggunakan teknik preprocessing yang dibahas sebelumnya (dijadikan dalam suatu fungsi "cleanText"

<p>[2]. Jalankan Voyant secara offline atau online di URL&nbsp;<a href="https://voyant-tools.org/" target="_blank">https://voyant-tools.org/</a></p>

<p>[3]. Upload file yang baru saja kita simpan (dataTweet_plain.txt).</p>


<h3 id="Penggunaan-Voyant-1:-WordClouds">Penggunaan Voyant 1: WordClouds</h3>

<ol>
	<li>Upload teks yang akan di analisa: hasil cluster/ suatu kategori/ topics / raw text.</li>
	<li>slider terms: mengkontrol banyaknya terms yang disertakan.</li>
	<li><strong>Summary </strong>(statistics)</li>
	<li><strong>Documents </strong>==&gt; add more</li>
	<li><strong>Phrases </strong>(n-grams like)</li>
	<li><strong>Export </strong>Visualisasi (kanan atas - pertama)</li>
	<li><strong>Options </strong>(kanan atas ke-3): Font, size, stopwords, whitelist</li>
	<li>&quot;?&quot; ==&gt; More Help</li>
</ol>

<p>&nbsp;</p>


<h3 id="Penggunaan-Voyant-2:-Bubbles">Penggunaan Voyant 2: Bubbles</h3>

<ol>
	<li>Upload teks yang akan di analisa: hasil cluster/ suatu kategori/ topics / raw text.<br />
	Atau file yang sudah terupload sebelumnya</li>
	<li>&nbsp;Klik tanda 4-kotak kecil (kanan atas ke-3)</li>
	<li>Pilih Visualization Tools ==&gt; Bubbles</li>
	<li>Option: hanya stopwords</li>
	<li>Export: Hanya PNG</li>
</ol>

<p>&nbsp;</p>


<h3 id="Penggunaan-Voyant-3:-Word-Tree">Penggunaan Voyant 3: Word Tree</h3>

<ol>
	<li>Upload teks yang akan di analisa: hasil cluster/ suatu kategori/ topics / raw text.<br />
	Atau file yang sudah terupload sebelumnya</li>
	<li>Klik branch untuk expand</li>
</ol>


<h3 id="Penggunaan-Voyant-2:-Bubbles">Penggunaan Voyant 4: Links</h3>

<ol>
	<li>Upload teks yang akan di analisa: hasil cluster/ suatu kategori/ topics / raw text.<br />
	Atau file yang sudah terupload sebelumnya</li>
	<li>Visualization Tools ==&gt; Links</li>
	<li>Klik sembarang terms untuk expand</li>
</ol>


<h3 id="Penggunaan-Voyant-5:-Trends">Penggunaan Voyant 5: Trends</h3>

<ol>
	<li>Upload teks yang akan di analisa: hasil cluster/ suatu kategori/ topics / raw text.<br />
	Atau file yang sudah terupload sebelumnya</li>
	<li>Document Tools ==&gt; Trends</li>
	<li>.. Butuh preprocessing ...&nbsp;</li>
	<li>Data harus terurut waktu</li>
	<li>Berikut contohnya</li>
</ol>


<p><strong>Latihan :&nbsp;</strong></p>

<ol>
	<li>Crawl twitter dengan salah satu topik yang sedang trending saat ini di Indonesia.&nbsp;</li>
	<li>Tentukan beberapa Important user-nya (centrality analysis)</li>
	<li>Bentuk graph komunitasnya.</li>
	<li>Analisa graph diatas di gephi.</li>
	<li>Analisa graphnya di Voyant Tools.</li>
</ol>


# <center><font color="blue"> End of Module

<hr />